In [14]:

import numpy as np
import os
import GPUtil
# plotting images
import matplotlib.pyplot as plt
import cv2
import time




file_dir = './data/full_data'
class_labels = np.array((33, 34, 35, 36, 38, 39, 40, 0))

# obtain a list of files in the given directory
filelist_x = np.array(sorted(os.listdir(os.path.join(file_dir, 'train_color'))))
filelist_y = np.array(sorted(os.listdir(os.path.join(file_dir, 'train_label'))))

In [15]:
containers = [32, 128, 224, 416, 1024, 2016 ,3392, 4000]
for cont in containers:
    result_dir = './data/full_data/processedData/train_color'+str(cont)
    result_dir1 = './data/full_data/processedData/train_label'+str(cont)
    if not os.path.exists(result_dir):
        os.mkdir(result_dir)
    if not os.path.exists(result_dir1):
        os.mkdir(result_dir1)

In [20]:
#path where to store
storeat = './data/full_data/processedData'

#ini the different sizes to grump the img to
containers = [32, 128, 224, 416, 1024, 2016 ,3392, 4000]
containercount = [0,0,0,0,0,0,0,0]
# a counter of how many are smaler then 50x50
countToSmall = 0

#k from and kend to
k= 0
#kend = 10
kend = len(filelist_x)


start = time.time()
# run your code


#loops through all imiges of filelist_x
while k < kend:
    #get img and label
    file_x = np.array(cv2.cvtColor(cv2.imread(os.path.join(file_dir, 
                                                                'train_color', 
                                                                filelist_x[k])), 
                                                                cv2.COLOR_BGR2RGB))
    file_y =  np.array(cv2.imread(os.path.join(file_dir, 'train_label', filelist_y[k]), -1))        
    
    
    #get the labels of interest in the current image
    labels, index, counts = np.unique(file_y , return_index=True ,return_counts=True)
    labels2 = labels//1000
    labels3 = []
    countnotempty =0
    for j in range(len(labels2)):
        if labels2[j] in class_labels and labels2[j] !=0:
            labels3.append(labels[j])
            
    #the index counts how many img has been grumped from the original and then saves the img like x_index.jpg
    index = 0
    #loop though all labels of interest in the current img
    for label in labels3:
        #find where the img has the current label
        x = np.where(file_y == label)
        if x[1].size < 1 or x[0].size <1:
            continue
            
            
        #get the ranges x and y of the current label
        xrange = max(x[1])-min(x[1])
        yrange = max(x[0]-min(x[0]))
        
        #if it is to small dont grump it.
        minsize = 0
        if xrange>minsize and yrange>minsize:
            
            #check in which container it has to be stored, eg 124x124 img or 224x224 img
            currentContainer = 0
            for container in containers:
                if xrange <= container and yrange <= container:
                    currentContainer = container 
                    break
                    
            #gets the upper left corner of the label and makes it for some percent bigger
            perc = 0.1
            cords = [min(x[1])-int(xrange*perc), min(x[0])-int(xrange*perc)]

            #check if it out of bounds for left and top
            if cords[0] <0:
                cords[0]=0
            if cords[1] <0:
                cords[0]=0
                
            #sets where to start and where to end the cut
            xstart = cords[1]
            xend = cords[1]+currentContainer
            ystart = cords[0]
            yend = cords[0]+currentContainer
            
            #if out of bounds down and right set the start such that it in again
            if xend > file_y.shape[0]:
                xstart = xstart - (xend - file_y.shape[0])
                xend = file_y.shape[0]
            if yend > file_y.shape[1]:
                ystart = ystart-(yend - file_y.shape[1])
                yend = file_y.shape[1]
            
            #if the currentCountainer is the full image then cut the full image
            if currentContainer == 3392 or currentContainer == 4000:
                xstart = 0
                ystart = 0
                xend = file_y.shape[0]
                yend = file_y.shape[1] 
            
            #cut the real img 
            x_cut = np.copy(file_x[xstart:xend, ystart:yend])
            y_cut = np.copy(file_y[xstart:xend, ystart:yend])
            
            #set all labels to 255 which are not part of the current label
            #notin = [label, 255]
            #y_cut[np.logical_not(np.isin(y_cut, notin))] = 255
            
            #save the crump img if it is not empty
            if x_cut == []:
                print("x is empty")
            elif y_cut == []:
                print("Y is empty")
            else:
                #create file names
                x_filename = os.path.join(storeat, 'train_color'+str(currentContainer), filelist_x[k][0:-4]+ '_' + str(index) + '.jpg')
                y_filename = os.path.join(storeat, 'train_label'+str(currentContainer), filelist_y[k][0:-4]+ '_' + str(index) + '.png')
                #print(x_filename)
                #print(y_filename)
                #increase the file name index
                index +=1
                #save img
                cv2.imwrite(x_filename, x_cut)
                cv2.imwrite(y_filename, y_cut)
                
                containercount[containers.index(currentContainer)]+=1
#                 plt.figure(figsize = (15,12))
#                 plt.subplot(1,4,1)
#                 plt.imshow(file_x); plt.title('train_color')
#                 plt.subplot(1,4,2)
#                 plt.imshow(file_y); plt.title('train_label')
#                 plt.subplot(1,4,3)
#                 plt.imshow(x_cut); plt.title('train_colot_cut')
#                 plt.subplot(1,4,4)
#                 plt.imshow(y_cut); plt.title('train_label_cut')                
#                 plt.show()
                
        else: 
            
            countToSmall+=1
            print("to small xrange {}, yrange {}, counter is {}".format(xrange, yrange, countToSmall))    
                
    
    end = time.time()

    elapsed = end - start
    k += 1
    
    print("at {} from {}; container {} sum {}; time {},{} [m,s]; time p.img {}s; est fin {}".format(k, kend, containercount,
                                                                                                          sum(containercount), 
                                                                                                          int(elapsed/60), 
                                                                                                          int(elapsed%60), 
                                                                                                          round(elapsed/k,3),
                                                                                                         round(((kend-k)*(elapsed/k))/3600,3) ))







/home/pabloleo96/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:97: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
/home/pabloleo96/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:99: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


at 1 from 39221; container [4, 2, 0, 0, 0, 0, 0, 0] sum 6; time 0,0 [m,s]; 0.747 p.img ; est fin 8.138
at 2 from 39221; container [9, 2, 0, 0, 0, 0, 0, 0] sum 11; time 0,1 [m,s]; 0.735 p.img ; est fin 8.007
at 3 from 39221; container [13, 4, 0, 0, 0, 0, 0, 0] sum 17; time 0,2 [m,s]; 0.748 p.img ; est fin 8.149
at 4 from 39221; container [18, 4, 0, 0, 0, 0, 0, 0] sum 22; time 0,2 [m,s]; 0.744 p.img ; est fin 8.105


KeyboardInterrupt: 